## 데이터 수집

### json file 관련 함수 

- save_json : 데이터를 json으로 저장
- load_json : json 파일 불러오기
- 기본 경로 현재 폴더인 'data'로 지정 해둠.

In [3]:
import os
import json

def save_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def load_json(filename):
    if not os.path.exists(filename):
        return None
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

### 마켓컬리 상품 크롤링 - 목록/각 상품 url

- 큰 카테고리 : "대분류", 그 다음 카테고리 : "소분류" 라고 정의
- "대분류" url -> CATEGORY_IDS 리스트에 url 마지막 숫자 3자리를 저장해놓음. for문으로 직접 넣어가며 반복.
- "소분류" url -> 그리드(Xpath 로 지정)에 있는 것들 하나씩 눌러가며 for문으로 반복. 단, 2번째부터 시작. 1번째는 항상 전체보기로 생략함.(소분류가 무엇인지 알 수 없음)
- 상품 url -> 최종적으로 분류된 url에 있는 상품 url들 모두 크롤링하여 저장. page=1 에서 page+=1 하다가 상품이 없다고 뜨면 중지로 페이지 모두 순회.
- 상품명 -> 상품 url로 들어가 이름 부분 크롤링

In [ ]:
import time
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] : %(message)s',
    handlers=[logging.StreamHandler()]
)

CATEGORY_IDS = ['722', '251','907', '908', '909', '910', '911', '912', '913', '914', '383', '249', '915', '018', '032']

# 로그인 쿠키
def login_and_save_cookies():
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        driver.get("https://www.kurly.com/member/login")
        logging.info("브라우저가 열렸습니다. 로그인 후 콘솔에 Enter를 입력하세요.")
        input("로그인 후 Enter 키를 누르세요...")

        cookies = driver.get_cookies()
        save_json(cookies, "cookies_kurly.json")
        logging.info("쿠키 저장 완료")

    finally:
        driver.quit()

# 크롤링
def crawl_products_kurly(cookies):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument("window-size=1920,1080")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    all_products = []
    error_products = []
    
    try:
        driver.get("https://www.kurly.com")
        time.sleep(2)

        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            if 'domain' in cookie:
                del cookie['domain']
            driver.add_cookie(cookie)

        driver.get("https://www.kurly.com/main")
        time.sleep(2)

        for category_id in CATEGORY_IDS:
            main_base_url = f'https://www.kurly.com/categories/{category_id}'
            driver.get(main_base_url)
            time.sleep(2)

            try:
                main_category = driver.find_element(By.XPATH, '//*[@id="container"]/h3').text.strip()
                logging.info(f'대분류({category_id}) - {main_category}')

                sub_category_count = len(driver.find_elements(By.CSS_SELECTOR, '#container > ul > li'))
                for i in range(2, sub_category_count + 1):
                    try:
                        sub_xpath = f'//*[@id="container"]/ul/li[{i}]/a'
                        sub_element = driver.find_element(By.XPATH, sub_xpath)
                        sub_category = sub_element.text.strip()
                        logging.info(f'소분류: {sub_category}')

                        sub_element.click()
                        time.sleep(2)

                        sub_base_url = driver.current_url.split('&page=')[0]
                        page = 1

                        while True:
                            page_url = f"{sub_base_url}&page={page}"
                            driver.get(page_url)
                            time.sleep(2)

                            if "상품이 없습니다" in driver.page_source:
                                logging.info("더 이상 상품 없음.")
                                break

                            logging.info(f'{page}페이지 수집 중...')

                            if sub_category.startswith("더퍼플"):
                                product_xpath_base = '//*[@id="container"]/div[3]/div[2]/div[2]/a'
                            else:
                                product_xpath_base = '//*[@id="container"]/div[2]/div[2]/div[2]/a'

                            product_elements = driver.find_elements(By.XPATH, product_xpath_base)

                            for idx in range(len(product_elements)):
                                try:
                                    product_elements = driver.find_elements(By.XPATH, product_xpath_base)
                                    product = product_elements[idx]
                                    driver.execute_script("arguments[0].scrollIntoView();", product)
                                    product.click()
                                    time.sleep(2)

                                    product_url = driver.current_url
                                    product_name = driver.find_element(
                                        By.XPATH,
                                        '//*[@id="product-atf"]/section/div[1]/div[1]/div[2]/h1'
                                    ).text.strip()

                                    # 기존에 있는지 확인(상품 URL 기준)
                                    existing = next((item for item in all_products if item['url'] == product_url), None)

                                    if existing:
                                        if sub_category not in existing['소분류']:
                                            existing['소분류'].append(sub_category)
                                    else:
                                        all_products.append({
                                            '상품명': product_name,
                                            'url': product_url,
                                            '대분류': main_category,
                                            '소분류': [sub_category]
                                        })

                                    driver.back()
                                    time.sleep(2)

                                except Exception as e:
                                    logging.warning(f'상품 오류: {e}')

                                    # 크롤링 오류 상품 url 저장
                                    error_url = driver.current_url
                                    if error_url not in error_products:
                                        error_products.append(error_url)
                                    driver.get(page_url)
                                    time.sleep(1)

                            page += 1

                        # 소분류 하나 끝날 때 저장
                        save_json(all_products, "crawling_products_kurly.json")
                        save_json(error_products, "crawling_error_products_kurly.json")
                        logging.info(f'현재까지 {len(all_products)}개 저장/{len(error_products)}개 오류')


                        driver.get(main_base_url)
                        time.sleep(2)

                    except Exception as e:
                        logging.error(f'소분류 {i} 오류: {e}')
                        driver.get(main_base_url)
                        time.sleep(2)

            except Exception as e:
                logging.error(f'대분류 {category_id} 오류: {e}')

    finally:
        driver.quit()

    return all_products

# 실행
if __name__ == "__main__":
    login_and_save_cookies()  
    cookies = load_json("cookies_kurly.json")  
    result = crawl_products_kurly(cookies)     
    logging.info(f"전체 크롤링 완료 - 총 {len(result)}개 상품 저장")

- 소분류 "와인위스키용품", "커피용품", "커피머신"은 식품에 해당하지 않아 제외

- 크롤링 오류 url로 다시 진행하려고 했으나 대분류/소분류 불가. 따라서 직접 추가

In [4]:
# 기존 데이터 불러오기
filepath = "crawling_products_kurly.json"
data = load_json(filepath)

# 추가할 상품들
new_items = [
    {
        "상품명": "[Kim's butcher] 미국산 돼지고기 샤브샤브 500g(냉동)",
        "url": "https://www.kurly.com/goods/1001101255",
        "대분류": "정육·가공육·달걀",
        "소분류": [
            "수입산 돼지고기·양고기"
        ]
    },
    {
        "상품명": "[미트클레버] 한돈 떡갈비 (1개입/2개입)",
        "url": "https://www.kurly.com/goods/5153270",
        "대분류": "정육·가공육·달걀",
        "소분류": [
            "돈까스·떡갈비·함박"
        ]
    }
]

# 붙이기
data.extend(new_items)

# 저장
save_json(data, filepath)
print(f"에러 상품 추가. 총 {len(data)}개 항목 저장됨.")


에러 상품 추가. 총 15717개 항목 저장됨.


### 마켓컬리 상품 크롤링 - 상세정보 (이미지 url/가격/세부정보)
- "이미지" url -> CSS_SELECTOR로 "#product-atf img"를 지정하여 상품 상세 페이지에서 대표 이미지 URL을 수집
- 상세정보 추출 -> 상세 항목들이 <li> 태그로 구성되어 있다는 점을 활용해 li:nth-child(n) 형태로 순차적으로 접근,각 dt(항목 이름)와 dd(항목 값)를 쌍으로 추출 및 details 딕셔너리에 저장
- 가격 추출 -> 상품의 가격 정보를 추출할 때, 단일 옵션과 다중 옵션 여부 및 할인 여부에 따라 HTML 구조가 달라 이를 구분하여 처리, label 텍스트를 기반으로 옵션 유형을 판별, 각 상황에 맞는 방식으로 정가 및 할인가 수집
- 주류 카테고리의 성인인증때문에 로그인 쿠키 설정

In [ ]:
import time
import copy
import logging

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] : %(message)s',
    handlers=[logging.StreamHandler()]
)    

# 로그인 후 쿠키 저장
def login_and_save_cookies():
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        driver.get("https://www.kurly.com/member/login")
        logging.info("로그인 후 Enter 키를 누르세요.")
        input("로그인 후 Enter 키를 누르세요...")

        cookies = driver.get_cookies()
        save_json(cookies, "cookies_kurly.json")
        logging.info("쿠키 저장 완료")
    finally:
        driver.quit()

#쿠키적용
def apply_cookies(driver, cookie_file="cookies_kurly.json"):
    cookies = load_json(cookie_file)
    if not cookies:
        logging.warning("쿠키 파일이 없습니다.")
        return

    driver.get("https://www.kurly.com")  # 먼저 도메인 접속
    time.sleep(2)
    for cookie in cookies:
        if 'sameSite' in cookie:
            del cookie['sameSite']
        if 'domain' in cookie:
            del cookie['domain']
        driver.add_cookie(cookie)
    driver.refresh()  # 쿠키 적용 후 새로고침
    time.sleep(2)
    logging.info("쿠키 적용 완료, 로그인 상태 유지됨.")


# 에러 url 저장 함수
def error_product_url(error_list, url):
    if url not in error_list:
        error_list.append(url)

# 이미지 저장 함수
def crawl_products_image(driver, product, error_products):

    try:
        driver.get(product["url"])
        time.sleep(1)  

        img_element = driver.find_element(By.CSS_SELECTOR, "#product-atf img")
        product["이미지"] = img_element.get_attribute("src")

    except Exception as e:
        logging.warning(f"[이미지] 크롤링 실패 : {e}")
        error_product_url(error_products, product["url"])

# 상세정보 추출 함수
def crawl_products_details(driver, product,error_products):
    details = {k: None for k in ["배송", "판매자", "포장타입", "판매단위", "중량/용량", "알레르기정보", "소비기한(또는 유통기한)정보", "안내사항"]}

    index = 1
    while True:
        try:
            detail_name = driver.find_element(By.CSS_SELECTOR, f"#product-atf > section > ul > li:nth-child({index}) > dt").text.strip()
            detail_info = driver.find_element(By.CSS_SELECTOR, f"#product-atf > section > ul > li:nth-child({index}) > dd").text.strip()
            if detail_name in details:
                details[detail_name] = detail_info
            index += 1

        except NoSuchElementException:
            break

        except Exception as e:
            logging.warning(f"[기타 정보] 크롤링 실패 : {e}")
            error_product_url(error_products, product["url"])
            break

    return details


# 가격 저장 함수
def crawl_products_price(driver, wait, product, error_products):
    driver.get(product["url"])
    time.sleep(1)

    try:
        common_info = crawl_products_details(driver, product,error_products)
        labels = driver.find_elements(By.CSS_SELECTOR, "dt.epzddad1")
        label_name = [dt.text.strip() for dt in labels]

        # "상품준비중입니다" 같은 버튼 텍스트도 label_name에 추가
        try:
            span_text = driver.find_element(By.CSS_SELECTOR,
                "#product-atf > section > div.css-1bp09d0.e17iylht1 > div.css-gnxbjx.e10vtr1i2 > div > button > span"
            ).text.strip()
            
            if span_text:
                label_name.append(span_text)  # 버튼 텍스트도 label_name에 추가
        except:
            pass  # 해당 버튼 없으면 그냥 무시

        results = []

        # 단일 상품
        if "상품선택" in label_name or "구매수량" in label_name or "상품준비중입니다" in label_name:
            p = copy.deepcopy(product)
            p["종류"] = None

            try:
                try:
                    driver.find_element(By.XPATH, '//*[@id="product-atf"]/section/h2/span[3]')
                    span3_exists = True
                except:
                    span3_exists = False

                if span3_exists:
                    p["정가"] = driver.find_element(By.XPATH, '//*[@id="product-atf"]/section/span/span').text.strip()
                    p["할인가"] = driver.find_element(By.XPATH, '//*[@id="product-atf"]/section/h2/span[2]').text.strip()
                else:
                    p["정가"] = driver.find_element(By.XPATH, '//*[@id="product-atf"]/section/h2/span[1]').text.strip()
                    p["할인가"] = None

            except Exception as e:
                p["정가"] = ""
                logging.warning(f"[정가 추출 오류] {e}")

            p.update(common_info)
            results.append(p)

        # 다중 상품
        elif "구매 수량" in label_name:
            click_target = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                "#product-atf > section > div.css-1bp09d0.e17iylht1 > div.css-2lvxh7.e1qy0s5w0 > li > dd > div > div:nth-child(1) > div > div > div")))
            click_target.click()
            time.sleep(1)

            index = 1
            while True:
                try:
                    option_name = wait.until(EC.presence_of_element_located(
                        (By.XPATH, f'//*[@id="menu-"]/div[3]/ul/li[{index}]/button/div/div[1]/span'))).text.strip()
                    try:
                        price_text = driver.find_element(By.XPATH, f'//*[@id="menu-"]/div[3]/ul/li[{index}]/button/div/div[2]/div[1]').text.strip()
                    except:
                        price_text = None
                    try:
                        discount_text = driver.find_element(By.XPATH, f'//*[@id="menu-"]/div[3]/ul/li[{index}]/button/div/div[2]/div[2]').text.strip()
                    except:
                        discount_text = None

                    p = copy.deepcopy(product)
                    p["종류"] = option_name
                    p["정가"] = price_text
                    p["할인가"] = discount_text
                    p.update(common_info)
                    results.append(p)
                    index += 1
                    time.sleep(1.0)
                except:
                    break

        return results

    except Exception as e:
        logging.warning(f"[ERROR] {product['url']}: {e}")
        error_url = driver.current_url
        if error_url not in error_products:
            error_products.append(error_url)
        return []


# 메인 실행 함수
def main():
    products = load_json("crawling_products_kurly.json")
    if products is None:
        logging.error("제품 목록 JSON 파일이 존재하지 않습니다.")
        return

    category_list = [
        "채소", "과일·견과·쌀", "수산·해산·건어물", "정육·가공육·달걀", "국·반찬·메인요리", "간편식·밀키트·샐러드", 
        "면·양념·오일", "생수·음료", "커피·차", "간식·과자·떡", "베이커리", "유제품", "건강식품", "와인·위스키·데낄라", "전통주"
        ]

    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 10)

    try:
        apply_cookies(driver, "cookies_kurly.json")

        for category in category_list:
            logging.info(f"=== [START] 대분류: {category} ===")

            filtered_products = [p for p in products if p.get("대분류") == category]
            error_products = load_json("error_products_kurly.json") or []

            count = 0
            while count < len(filtered_products):
                product = filtered_products[count]

                # 이미지 크롤링
                crawl_products_image(driver, product, error_products)

                # 상세 정보 크롤링
                details = crawl_products_details(driver, product, error_products)
                product.update(details)

                # 가격 크롤링
                price_results = crawl_products_price(driver, wait, product, error_products)

                if len(price_results) == 1:
                    # 단일 옵션: product 업데이트 후 그대로 진행
                    filtered_products[count].update(price_results[0])
                    count += 1

                elif len(price_results) > 1:
                    # 다중 옵션: 원본 상품 제거 후 옵션별 상품 추가
                    filtered_products.pop(count)
                    for idx, opt_product in enumerate(price_results):
                        filtered_products.insert(count + idx, opt_product)
                    count += len(price_results)

                else:
                    # 실패 시 건너뛰고 다음 상품
                    count += 1

                # 200개 단위 저장
                if count % 200 == 0 or count == len(filtered_products):
                    save_json(filtered_products, "crawling_products_kurly.json")
                    save_json(error_products, "error_products_kurly.json")
                    logging.info(f"{count}개 저장 완료.")

                time.sleep(1)

            logging.info(f"[{category}] 상품 크롤링 완료. 총 {len(filtered_products)}개 저장됨.")

    finally:
        driver.quit()

    logging.info(f"전체 크롤링 완료.")




if __name__ == "__main__":
    login_and_save_cookies()
    main()

### 에러 상품 삭제
- 에러로 "종류" key 값 자체가 없는 경우 상품 자체 삭제
- 위 에러는 상품이 마켓컬리에서 아예 판매 중지된 상품 + 예외적으로 크롤링이 되지 않으나 원인이 보이지 않는 소수의 상품

In [ ]:
category_list = [
        "채소", "과일·견과·쌀", "수산·해산·건어물", "정육·가공육·달걀", "국·반찬·메인요리", "간편식·밀키트·샐러드", 
        "면·양념·오일", "생수·음료", "커피·차", "간식·과자·떡", "베이커리", "유제품", "건강식품", "와인·위스키·데낄라", "전통주"
        ]

for category in category_list:
    products = load_json(f"crawling_products_kurly_{category}.json")

    # "종류" 키가 없는 아이템 제거
    products = [p for p in products if "종류" in p]

    save_json(products, f"crawling_products_kurly_{category}.json")
    logging.info(f"[{category}] - 총 {len(products)}개의 상품")

### 마켓컬리 상품 크롤링 - 후기
- 각 상품마다 15개씩 후기 크롤링
- "종류"가 나눠져있는 경우 종류별로 다르게 수집

In [ ]:
import time
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 드라이버 설정
options = Options()
options.add_argument("--headless")  
options.add_argument("window-size=1920,1080")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

# 상품명에서 옵션 이름으로 변환하는 함수
def get_clean_product_name(full_name):
                # (품절) 제거
                if full_name.startswith("(품절)"):
                    full_name = full_name.replace("(품절)", "").strip()
                # [브랜드] 제거
                if full_name.startswith('[') and ']' in full_name:
                    full_name = full_name.split(']', 1)[1].strip()
                return full_name.strip()

# 옵션에서 옵션 이름만 추출하는 함수
def get_clean_option_name(option_html):
    soup = BeautifulSoup(option_html, "html.parser")
    price_span = soup.find("span", class_="css-487zd1 enznrj40")
    # 가격 span 제거
    if price_span:
        price_span.extract()
    clean_name = soup.get_text(strip=True)
    return clean_name

# 에러 항목
error_list=[]

# 크롤링
for category in category_list:

    products = load_json(f"crawling_products_kurly_{category}.json")

    # 시작
    for index,product in enumerate(products, start=1):
        driver.get(product["url"])
        driver.execute_script("window.scrollTo(0, 600);") # 후기 탭이 보이도록 아래로 스크롤
        time.sleep(0.3)

        # 후기 탭 클릭
        try:
            review_tab = driver.find_element(By.CSS_SELECTOR, "#top > div.css-n48rgu.ex9g73v0 > div.css-16c0d8l.e1brqtzw0 > nav > ul > li:nth-child(3) > a")
            review_tab.click()
            time.sleep(0.3)
        except Exception as e:
            error_list.append(product)
            logging.warning(f"후기 탭 클릭 실패: {e}")
            
        ## 1. 종류가 없는 경우
        if product["종류"] is None:
            try:
                driver.execute_script("window.scrollTo(0, 600);")
                time.sleep(0.5)

                # "후기수"
                try:
                    review_count = driver.find_element(By.CSS_SELECTOR, '#top > div.css-n48rgu.ex9g73v0 > div.css-16c0d8l.e1brqtzw0 > nav > ul > li:nth-child(3) > a > span.count')
                    product["후기수"] = review_count.text.strip()

                except:
                    product["후기수"] = "0"

                # "후기"
                product["후기"] = []
                collected = 0

                while collected < 15:
                    for i in range(4, 14):  # div[4] ~ div[13]
                        if collected >= 15:
                            break

                        base_xpath = f'//*[@id="review"]/section/div[2]/div[{i}]'

                        # '베스트' 태그 확인
                        try:
                            best_tag = driver.find_element(By.XPATH, base_xpath + '/div/div/div')
                            if '베스트' in best_tag.text:
                                continue
                        except:
                            pass

                        # 닉네임 '컬*' 확인
                        try:
                            nickname_tag = driver.find_element(By.XPATH, base_xpath + '/div/div')
                            if nickname_tag.text.strip()=="컬*":
                                continue
                        except:
                            pass

                        # 실제 리뷰 내용
                        try:
                            review_elem = driver.find_element(By.XPATH, base_xpath + '/article/div/p')
                            review_text = review_elem.text.strip()
                            if review_text:
                                product["후기"].append(review_text)
                                collected += 1
                        except:
                            continue

                    # 다음 페이지 버튼 
                    if collected < 15:
                        try:
                            next_button = driver.find_element(By.CSS_SELECTOR, '#review > section > div:nth-child(3) > div.css-jz9m4p.ebs5rpx3 > button.css-1orps7k.ebs5rpx0')
                            next_button.click()
                            time.sleep(0.3)

                        except Exception as e:
                            break

            except Exception as e:
                error_list.append(product)
                logging.error(f"오류 발생: {e}")
                
        ## 2. 종류가 있는 경우
        else:
            # 옵션 선택 버튼 클릭
            try:
                # 옵션 버튼
                option_button = driver.find_element(By.XPATH, '//*[@id="review"]/section/div[2]/div[2]/button/span')

                # 버튼을 화면 중앙에 오도록 스크롤
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", option_button)
                time.sleep(0.3)

                # ActionChains를 이용해 클릭
                actions = ActionChains(driver)
                actions.move_to_element(option_button).click().perform()
                time.sleep(0.3)

            except Exception as e:
                error_list.append(product)
                logging.error(f"옵션 선택 버튼 클릭 실패: {e}")

            # 옵션 클릭
            try:
                target_product_name = get_clean_product_name(product["종류"])

                base_xpath_prefix = '//*[@id="review"]/section/div[2]/div[2]/div/div[2]/ul/li'

                option_count = 0
                option_elements = []

                i = 1
                while True:
                    current_xpath = f"{base_xpath_prefix}[{i}]"
                    try:
                        option_elem = driver.find_element(By.XPATH, current_xpath)
                        option_elements.append(option_elem)
                        option_count += 1
                        i += 1
                    except:
                        break

                matched_index = None
                for idx, option_elem in enumerate(option_elements, start=1):
                    option_span_xpath = f'{base_xpath_prefix}[{idx}]/button/div/span'
                    try:
                        option_span_elem = driver.find_element(By.XPATH, option_span_xpath)
                        option_html = option_span_elem.get_attribute('outerHTML')
                    except:
                        continue

                    clean_option_name = get_clean_option_name(option_html)

                    if clean_option_name == target_product_name:
                        matched_index = idx
                        break
                    else:
                        pass

                if matched_index is not None:
                    option_button_xpath = f'{base_xpath_prefix}[{matched_index}]/button'
                    try:
                        option_button = driver.find_element(By.XPATH, option_button_xpath)
                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", option_button)
                        time.sleep(0.3)
                        actions = ActionChains(driver)
                        actions.move_to_element(option_button).click().perform()
                        time.sleep(0.3)
                    except Exception as e:
                        logging.error(f"옵션 클릭 실패: {e}")
                else:
                    error_list.append(product)
                    logging.warning(f"옵션 '{target_product_name}'에 해당하는 항목을 찾지 못했습니다.")

            except Exception as e:
                logging.error(f"옵션 클릭 관련 에러: {e}")

            # 후기 보기 버튼 클릭 
            try:
                review_button_xpath = '//*[@id="review"]/section/div/div[2]/div/footer/button'
                review_text_xpath = '//*[@id="review"]/section/div/div[2]/div/footer/button/p'

                review_button = driver.find_element(By.XPATH, review_button_xpath)
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", review_button)
                time.sleep(0.3)
                review_button.click()
                time.sleep(0.3)

            except Exception as e:
                logging.error(f"후기 보기 버튼 클릭 실패: {e}")

            # "후기수"
            try:
                review_count = driver.find_element(By.CSS_SELECTOR, '#top > div.css-n48rgu.ex9g73v0 > div.css-16c0d8l.e1brqtzw0 > nav > ul > li:nth-child(3) > a > span.count')
                product["후기수"] = review_count.text.strip()

            except:
                product["후기수"] = "0"

            # "후기"
            product["후기"] = []
            collected = 0

            while collected < 15:
                for i in range(4, 14):  # div[4] ~ div[13]
                    if collected >= 15:
                        break

                    base_xpath = f'//*[@id="review"]/section/div[2]/div[{i}]'

                    # '베스트' 태그 확인
                    try:
                        best_tag = driver.find_element(By.XPATH, base_xpath + '/div/div/div')
                        if '베스트' in best_tag.text:
                            continue
                    except:
                        pass

                    # 닉네임 '컬*' 확인
                    try:
                        nickname_tag = driver.find_element(By.XPATH, base_xpath + '/div/div')
                        if nickname_tag.text.strip()=="컬*":
                            continue
                    except:
                        pass

                    # 실제 리뷰 내용
                    try:
                        review_elem = driver.find_element(By.XPATH, base_xpath + '/article/div/p')
                        review_text = review_elem.text.strip()
                        if review_text:
                            product["후기"].append(review_text)
                            collected += 1
                    except:
                        continue

                # 다음 페이지 버튼 
                if collected < 15:
                    try:
                        next_button = driver.find_element(By.CSS_SELECTOR, '#review > section > div:nth-child(3) > div.css-jz9m4p.ebs5rpx3 > button.css-1orps7k.ebs5rpx0')
                        next_button.click()
                        time.sleep(0.3)

                    except Exception as e:
                        break

        # 저장
        if (index % 50) == 0 or index==len(products):
            save_json(products, f"crawling_products_kurly_{category}.json")
            save_json(error_list, f"crawling_products_kurly_{category}_error.json")
            logging.info(f"[{index}/{len(products)}]개 저장 완료")

    logging.info(f"[{category}] - 후기 크롤링 완료")

logging.info(f"전체 항목 후기 크롤링 완료")